# A4 - Common Analysis  

During the last two years we all have been experiencing a global pandemic. This has been tragic and disruptive to many countries and has taken a deep personal toll on many individuals and their families.  
One aspect that has been hard to miss in the last two years is the datafication of the pandemic. That is, many aspects of the individual toll of the pandemic have been collected, aggregated and re-represented as data. This datafication gives us the privilege to examine the pandemic from potentially many different perspectives to understand how it has changed lives and how it has changed society. To be honest, we are actually at the very beginning of understanding and comprehending these impacts.  
During our Course Project we are going to begin taking a look at some of the social aspects of the pandemic by conducting a human centered data science analysis of some available COVID-19 data. In this Assignment A4: Common Analysis, every student in the course will work from the same datasets. Students will be assigned to analyze data for one specific County of the United States.

## Step 0 - Data Acquisition
The common analysis research question will require several different datasets. You will need:  
- The RAW_us_confirmed_cases.csv file from the [Kaggle repository of John Hopkins University COVID-19 data.](https://www.kaggle.com/antgoldbloom/covid19-data-from-john-hopkins-university?select=RAW_us_confirmed_cases.csv)  
- The [CDC dataset](https://data.cdc.gov/Policy-Surveillance/U-S-State-and-Territorial-Public-Mask-Mandates-Fro/62d6-pm5i) of masking mandates by county.  
- The New York Times mask compliance [survey data.](https://github.com/nytimes/covid-19-data/tree/master/mask-use)  

The majority of this data is by US County by Day. The mask compliance is a single shot estimator that gives you a compliance estimate for every County in the US. You should carefully review the data descriptions that accompany these datasets. They each have some interesting caveats. As well, some of them are explicit with regard to the way you should interpret missing data.  

My assignment:  

| County | State | Area_km^2| Area_mi^2 | Population_2020_Census | Population_Increase_from_2010 | County Seat |  
| ------ | ----- | -------- | --------- | ---------------------- | ----------------------------- | ----------- |  
| Orange | California | 2,047.56 | 790.57 | 3,186,989 | 176,757 | Santa Ana |  

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [4]:
# Set up
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Show all columns
pd.set_option('display.max_columns', None)

# Set filepaths
RAW_DATA_PATH = '../data_raw/'
CLEAN_DATA_PATH = '../data_clean/'

In [12]:
# Import data
cases_raw = pd.read_csv(RAW_DATA_PATH + 'RAW_us_confirmed_cases.csv/RAW_us_confirmed_cases.csv')
mandates_raw = pd.read_csv(RAW_DATA_PATH + 'U.S._State_and_Territorial_Public_Mask_Mandates_From_April_10__2020_through_August_15__2021_by_County_by_Day.csv')
masks_raw = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv')
print(cases_raw.head(10))
print(mandates_raw.head(10))
print(masks_raw.head(10))

  Province_State    Admin2       UID iso2 iso3  code3    FIPS Country_Region  \
0        Alabama   Autauga  84001001   US  USA    840  1001.0             US   
1        Alabama   Baldwin  84001003   US  USA    840  1003.0             US   
2        Alabama   Barbour  84001005   US  USA    840  1005.0             US   
3        Alabama      Bibb  84001007   US  USA    840  1007.0             US   
4        Alabama    Blount  84001009   US  USA    840  1009.0             US   
5        Alabama   Bullock  84001011   US  USA    840  1011.0             US   
6        Alabama    Butler  84001013   US  USA    840  1013.0             US   
7        Alabama   Calhoun  84001015   US  USA    840  1015.0             US   
8        Alabama  Chambers  84001017   US  USA    840  1017.0             US   
9        Alabama  Cherokee  84001019   US  USA    840  1019.0             US   

         Lat      Long_  ... 10/22/21  10/23/21  10/24/21  10/25/21  10/26/21  \
0  32.539527 -86.644082  ...     9967 

- FIPS is the key for each unique county. It is stored as 1 number in the `cases_raw` dataset but two values in the `mandate_raw` data set. When `FIPS_State` is single-digit then 0 is appended to the end before concatenating with `FIPS_County`. If `FIPS_County` is single digit as well, then 0 is appended to the beginning. Ex: Alabama Autauga County has state FIP 1 and county FIP 1, so it becomes 1001 for the combined FIP.
- For time-series analysis each date should be in one column, with the value in another. Long vs. wide data.  

In [74]:
# identify Orange County, CA FIPs code
fip = cases_raw.loc[(cases_raw['Province_State'] == 'California') & (cases_raw['Admin2'] == 'Orange'), 'FIPS'].values[0]
# print(fip)

# Create new combined FIPS column for mandates data set.
# If state code is single digit, then adds trailing 0
mandates_raw['FIPS'] = mandates_raw['FIPS_County'] + mandates_raw['FIPS_State'].apply(lambda x: x * 100 if x > 9 else x * 1000)

# Convert mandates_raw column into datetime
mandates = mandates_raw.copy()
mandates['date'] = pd.to_datetime(mandates['date'], format = '%m/%d/%Y')

6059.0


In [ ]:
# Create combined data set
covid_df = cases